In [5]:
import numpy as np
import random as pyrandom
from parse_input import *


# atoms.txt Содержит информацию об атомах, в колонках следующие данные:
# ID, x, y, z, сигма, эпсилон, заряд, 
# радиус Ван-дер-Ваальса, объём и еще две колонки типа строки; 
# радиус и объем и эти две колонки не используются далее
atoms = rd('atoms.txt', 1, 9)  # вместо 9 можно было и 7 взять

# bonds.txt Содержит информацию о связях, структура файла:
# ID1, ID2, Kr, r0
bonds = rd('bonds.txt', 2)

# angles.txt Содержит информацию об углах, структура файла:
# ID1, ID2, ID3, Kt, theta0
angles = rd('angles.txt', 3)

# dihedrals.txt Содержит информацию о двугранных углах, структура:
# ID1, ID2, ID3, ID4, V1, V2, V3, V4
# Все константы V1, V2, V3, V4 разделены на два, 
# при вычислении энергии делить их на два НЕ НАДО.
dihedrals = rd('dihedrals.txt', 4)

# neighbours.txt, two_bonds_neigh.txt, three_bonds_neigh.txt
# Содержат информацию о соседях непосредственных, 
# второго и третьего порядка соответственно. 
# У всех одна и та же структура:
# ID    id_1 id_2 ... id_n
# ID – индекс атома, id_1 id_2 ... id_n – индексы его соседей.
neighbours = rd('neighbours.txt')
two_bonds_neigh = rd('two_bonds_neigh.txt')

three_bonds_neigh = rd('three_bonds_neigh.txt')


In [1]:
import math

initial_coef = np.array([1.0, 1.0, 1.0],  dtype=np.float64)
print("Initial structure: a   = %f,   b = %f,   c = %f" % (initial_coef[0], initial_coef[1], initial_coef[2]))

parabola_arg = np.array([1.0, 2.0, 3.0],  dtype=np.float64)
print("Initial arguments: x_1 = %f, x_2 = %f, x_3 = %f" % (parabola_arg[0], parabola_arg[1], parabola_arg[2]))


def parabola(x):
    return initial_coef[0]*(x**2) + initial_coef[1]*x + initial_coef[2]


candidates = np.array([ parabola(parabola_arg[0]), parabola(parabola_arg[1]), parabola(parabola_arg[2])],  dtype=np.float64)    
print("Initial parabola value: d_1 = %f, d_2 = %f, d_3 = %f" % (candidates[0], candidates[1], candidates[2]))
parabola_coefficients = {'a': 0.0, 'b': 0.0, 'c': 0.0}
# print(parabola_arg)
denominator = \
    parabola_arg[0] * np.power(parabola_arg[1], 2) - np.power(parabola_arg[0], 2) * parabola_arg[1] \
    - parabola_arg[0] * np.power(parabola_arg[2], 2) + parabola_arg[1] * np.power(parabola_arg[2], 2) \
    + np.power(parabola_arg[0], 2) * parabola_arg[2] - np.power(parabola_arg[1], 2) * parabola_arg[2]

parabola_coefficients['a'] = \
    - parabola_arg[1] * candidates[0] + parabola_arg[2] * candidates[0] \
    + parabola_arg[0] * candidates[1] - parabola_arg[2] * candidates[1] \
    - parabola_arg[0] * candidates[2] + parabola_arg[1] * candidates[2]

parabola_coefficients['b'] = \
    np.power(parabola_arg[1], 2) * candidates[0] - np.power(parabola_arg[2], 2) * candidates[0] \
    - np.power(parabola_arg[0], 2) * candidates[1] + np.power(parabola_arg[2], 2) * candidates[1] \
    + np.power(parabola_arg[0], 2) * candidates[2] - np.power(parabola_arg[1], 2) * candidates[2]

parabola_coefficients['c'] = \
    parabola_arg[1] * np.power(parabola_arg[2], 2) * candidates[0] - np.power(parabola_arg[1], 2) * parabola_arg[2] * candidates[0] \
    - parabola_arg[0] * np.power(parabola_arg[2], 2) * candidates[1] + np.power(parabola_arg[0], 2) * parabola_arg[2] * candidates[1] \
    + parabola_arg[0] * np.power(parabola_arg[1], 2) * candidates[2] - np.power(parabola_arg[0], 2) * parabola_arg[1] * candidates[2]

print("Counted fraction value: a = %f/%f, b = %f/%f, c = %f/%f" % (parabola_coefficients['a'], denominator, parabola_coefficients['b'], denominator, parabola_coefficients['c'], denominator))
skip = False
for coef in parabola_coefficients:
    parabola_coefficients[coef] = parabola_coefficients[coef] / denominator
    if not math.isnan(parabola_coefficients[coef]):
        print("Resulting coefficients ", coef, ": ", parabola_coefficients[coef])
    else:
        skip = True


NameError: name 'np' is not defined

In [6]:
from output import *
from energy import *
from itertools import cycle

N = 5000  # 10000 # число итераций
l = len(atoms)  # число атомов в изучаемом белке
h = [0.02, 0.02, 0.02]  # размеры шагов по ортам в ангстремах
L = 3  # сколько шагов делаем вдоль каждого орта с учетом знака
S = 0
r = 10  # размер окрестности атома, по которой честно суммируем
M = 1000  # параметр, отвечающий за частоту вывода и сохранения информации
mas = []
tt = []
time_start = millis()

for k in range(0, N):
    atoms_old = atoms
    record = [0, 0, 0]
    num = pyrandom.randrange(0, l)
    nghb_d = nghb_dihedrals(dihedrals, num)
    nghb_a = nghb_angles(angles, num)
    nghb_b = nghb_bonds(bonds, num)
    nghb_nb = nghbnb(atoms, num, r)[0]
    nghbc = nghbnb(atoms, num, r)[1]
    [Cx, Cy, Cz] = coef_nb(atoms, num, nghbc)
    DE_elst = E_elst(atoms, neighbours, two_bonds_neigh, three_bonds_neigh, num) - \
              E_elst_m(atoms, neighbours, two_bonds_neigh, three_bonds_neigh, num, nghb_nb)

    E_old = E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh,
              three_bonds_neigh, num, nghb_d, nghb_a, nghb_b, nghb_nb, full=True)
    E_record = E_old

    candidates = np.array([0.0, 0.0, 0.0], dtype=np.float64)
    parabola_arg = np.array([0.0, 0.0, 0.0], dtype=np.float64)
    tmp = atoms_old[num][1]

    a = cycle([Cx, Cy, Cz])
    al = cycle([1, 2, 3])
    c = next(a)
    where = next(al)

    for indent in [-1, 0, 1]:
        print("atom: ", tmp, " ind: ", indent, " sum: ", np.float64(tmp + indent * h[0]))
        parabola_arg[1 + indent] = np.float64(tmp + indent * h[0])
        atoms[num][where] = atoms_old[num][where] + indent * h[0]
        candidates[1 + indent] = DE_elst + c * indent * h[0] + \
                                 E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh,
                                   three_bonds_neigh, num, nghb_d, nghb_a, nghb_b, nghb_nb, full=False)
    
    count_parabola = np.NaN not in candidates

    # print("Initial arguments: x_1 = %f, x_2 = %f, x_3 = %f" % (parabola_arg[0], parabola_arg[1], parabola_arg[2]))
    parabola_coefficients = {'a': 0.0, 'b': 0.0, 'c': 0.0}
    denominator = \
        parabola_arg[0] * np.power(parabola_arg[1], 2) - np.power(parabola_arg[0], 2) * parabola_arg[1] \
        - parabola_arg[0] * np.power(parabola_arg[2], 2) + parabola_arg[1] * np.power(parabola_arg[2], 2) \
        + np.power(parabola_arg[0], 2) * parabola_arg[2] - np.power(parabola_arg[1], 2) * parabola_arg[2] 

    parabola_coefficients['a'] = \
        - parabola_arg[1] * candidates[0] + parabola_arg[2] * candidates[0] \
        + parabola_arg[0] * candidates[1] - parabola_arg[2] * candidates[1] \
        - parabola_arg[0] * candidates[2] + parabola_arg[1] * candidates[2]

    parabola_coefficients['b'] = \
        np.power(parabola_arg[1], 2) * candidates[0] - np.power(parabola_arg[2], 2) * candidates[0] \
        - np.power(parabola_arg[0], 2) * candidates[1] + np.power(parabola_arg[2], 2) * candidates[1] \
        + np.power(parabola_arg[0], 2) * candidates[2] - np.power(parabola_arg[1], 2) * candidates[2]

    parabola_coefficients['c'] = \
        parabola_arg[1] * np.power(parabola_arg[2], 2) * candidates[0] - \
            np.power(parabola_arg[1], 2) * parabola_arg[2] * candidates[0] \
        - parabola_arg[0] * np.power(parabola_arg[2], 2) * candidates[1] + \
            np.power(parabola_arg[0], 2) * parabola_arg[2] * candidates[1] \
        + parabola_arg[0] * np.power(parabola_arg[1], 2) * candidates[2] - \
            np.power(parabola_arg[0], 2) * parabola_arg[1] * candidates[2]
    
    skip = False
    for coef in parabola_coefficients:
        parabola_coefficients[coef] = parabola_coefficients[coef] / denominator
        # if math.isnan(parabola_coefficients[coef]):
        #    skip = True
        # print("Resulting coefficients ", coef, ": ", parabola_coefficients[coef])
        # else:
        #    skip = True

    if np.NaN in parabola_arg:
        ind = np.where(parabola_arg != np.NaN)[0]
        if len(ind) != 1:
            if np.isnan(ind[0]):
                ind = 0
            else:
                ind = ind[0]
        parabola_min_arg = parabola_arg[ind]
    
    if not skip:
        parabola_min_arg = np.float64( (- parabola_coefficients['b']) / (2 * parabola_coefficients['a']))
        # atoms[num][1] = atoms_old[num][1] + parabola_min_arg
    
    print("Minimum of parabola: ", parabola_min_arg)
    
    E_new = parabola_coefficients['a'] * np.power(parabola_min_arg, 2) + \
            parabola_coefficients['b'] * parabola_min_arg + parabola_coefficients['c']
    print("Candidates: ", candidates[0], "\n\t", candidates[1], "\n\t", candidates[2])
    
    print("E_new: ", E_new, " E_old: ", E_old)
    if np.isnan(E_new):
        E_new = E_old
    
    print("Supposed minimum: ", E_new)
    if E_new > candidates[0] or E_new > candidates[1] or E_new > candidates[2] or np.isnan(E_new):
        print("Parabola is not at minimum.")
        E_new = np.min(candidates)
        print("Actual minimum: ", E_new)
        ind = np.where(candidates == E_new)[0]
        if len(ind) != 1:
            if np.isnan(ind[0]):
                ind = 0
            else:
                ind = ind[0]
        direction = int(ind)
        h[direction] += h[direction]
        atoms[num][direction] = atoms_old[num][direction] + parabola_min_arg * h[where]
    else:
        atoms[num][where] = atoms_old[num][where] + parabola_min_arg * h[where]
    
    print("\nE_new: ", E_new, "\tE_old: ", E_old, "\tDE_elst: ", DE_elst)
    #  "-0.2" чтобы не учитвать погрешности в аппроксимации
    if E_new < E_record:
        # record = [px, py, pz]
        E_record = E_new

    S = S + E_old - E_record
    mas.append(S)
    time_now = millis()
    ttm = (time_now - time_start) / 60000
    tt.append(ttm)

    # это просто печать результатов в файлы и отображения хода работы метода
    if k/M == int(k/M):
        # prp_atm(atoms, k)
        # os.system('./to-pdb atoms_out' + str(k) + '.csv output' + str(k) + '.pdb')
        time_now = millis()
        tm = (time_now - time_start)/60000
        print("################################")
        print("Число итераций {}; Энергия {}; Время (мин) {}".format(k, S, tm))
        print("################################\n")
        
print(S)

#     for px in range(-L, L + 1):
#         for py in range(-L, L + 1):
#             for pz in range(-L, L + 1):
#                 atoms[num][1] = atoms_old[num][1] + px * h[0]
#                 atoms[num][2] = atoms_old[num][2] + py * h[1]
#                 atoms[num][3] = atoms_old[num][3] + pz * h[2]
#                 E_new = DE_elst + Cx * px * h[0] + Cy * py * h[1] + Cz * pz * h[2] + \
#                         E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh,
#                           three_bonds_neigh, num, nghb_d, nghb_a, nghb_b, nghb_nb, full=False)
#                 #  "-0.2" чтобы не учитвать погрешности в аппроксимации
#                 if E_new < E_record - 2:
#                     record = [px, py, pz]
#                     E_record = E_new
#     atoms = atoms_old
# 
#     atoms[num][1] = atoms_old[num][1] + record[0] * h[0]
#     atoms[num][2] = atoms_old[num][2] + record[1] * h[1]
#     atoms[num][3] = atoms_old[num][3] + record[2] * h[2]
# 
#     S = S + E_old - E_record
#     mas.append(S)
#     time_now = millis()
#     ttm = (time_now - time_start) / 60000
#     tt.append(ttm)
#     # приводимый далее блок можно закомментировать
#     # это просто печать результатов в файлы и отображения хода работы метода
#     # if k/M == int(k/M):
#     #     prp_atm(atoms,k)
#     #     os.system('./to-pdb atoms_out' + str(k) + '.csv output' + str(k) + '.pdb')
#     #     time_now = millis()
#     #     tm = (time_now - time_start)/60000
#     #     print("Число итераций {}; Энергия {}; Время (мин) {}".format(k,S,tm))
# print(S)


('atom: ', 28.941, ' ind: ', -1, ' sum: ', 28.920999999999999)
('atom: ', 28.941, ' ind: ', 0, ' sum: ', 28.940999999999999)
('atom: ', 28.941, ' ind: ', 1, ' sum: ', 28.960999999999999)
('Minimum of parabola: ', 28.930820405909923)
('Candidates: ', -14.250691881187649, '\n\t', -14.252869750372422, '\n\t', -14.497580073572568)
('E_new: ', -14.221457212319365, ' E_old: ', -14.532964945943512)
('Supposed minimum: ', -14.221457212319365)
Parabola is not at minimum.
('Actual minimum: ', -14.497580073572568)
('\nE_new: ', -14.497580073572568, '\tE_old: ', -14.532964945943512, '\tDE_elst: ', 3.2673550708348458)
################################
Число итераций 0; Энергия 0.0; Время (мин) 0
################################



('atom: ', 16.346, ' ind: ', -1, ' sum: ', 16.326000000000001)
('atom: ', 16.346, ' ind: ', 0, ' sum: ', 16.346)
('atom: ', 16.346, ' ind: ', 1, ' sum: ', 16.366)
('Minimum of parabola: ', 16.328933479919705)
('Candidates: ', -12.963249429070075, '\n\t', -13.067765132913284, '\n\t', -13.468086132040344)
('E_new: ', -12.960066941435798, ' E_old: ', -13.363748450742577)
('Supposed minimum: ', -12.960066941435798)
Parabola is not at minimum.
('Actual minimum: ', -13.468086132040344)
('\nE_new: ', -13.468086132040344, '\tE_old: ', -13.363748450742577, '\tDE_elst: ', -2.5913940030725566)
################################
Число итераций 1; Энергия 0.104337681298; Время (мин) 0
################################



('atom: ', 27.052, ' ind: ', -1, ' sum: ', 27.032)
('atom: ', 27.052, ' ind: ', 0, ' sum: ', 27.052)
('atom: ', 27.052, ' ind: ', 1, ' sum: ', 27.071999999999999)
('Minimum of parabola: ', 27.040941362358883)
('Candidates: ', -1.528284123913437, '\n\t', -1.5371758918482605, '\n\t', -1.7140527655686348)
('E_new: ', -1.5114963205996901, ' E_old: ', -1.7436096467070132)
('Supposed minimum: ', -1.5114963205996901)
Parabola is not at minimum.
('Actual minimum: ', -1.7140527655686348)
('\nE_new: ', -1.7140527655686348, '\tE_old: ', -1.7436096467070132, '\tDE_elst: ', -9.8560588393542439)
################################
Число итераций 2; Энергия 0.104337681298; Время (мин) 0
################################



('atom: ', 23.393, ' ind: ', -1, ' sum: ', 23.373000000000001)
('atom: ', 23.393, ' ind: ', 0, ' sum: ', 23.393000000000001)
('atom: ', 23.393, ' ind: ', 1, ' sum: ', 23.413)
('Minimum of parabola: ', 23.382988582140342)
('Candidates: ', -29.516976574265737, '\n\t', -29.517758957394044, '\n\t', -30.888987997538216)
('E_new: ', -29.346062716213055, ' E_old: ', -31.164591574412523)
('Supposed minimum: ', -29.346062716213055)
Parabola is not at minimum.
('Actual minimum: ', -30.888987997538216)
('\nE_new: ', -30.888987997538216, '\tE_old: ', -31.164591574412523, '\tDE_elst: ', -19.063928645072952)
################################
Число итераций 3; Энергия 0.104337681298; Время (мин) 0
################################



('atom: ', 13.804, ' ind: ', -1, ' sum: ', 13.784000000000001)
('atom: ', 13.804, ' ind: ', 0, ' sum: ', 13.804)
('atom: ', 13.804, ' ind: ', 1, ' sum: ', 13.824)
('Minimum of parabola: ', 13.7743635940634)
('Candidates: ', -135.94593120235967, '\n\t', -136.10996349642585, '\n\t', -136.44106535619457)
('E_new: ', -135.92653541336767, ' E_old: ', -136.27747155920372)
('Supposed minimum: ', -135.92653541336767)
Parabola is not at minimum.
('Actual minimum: ', -136.44106535619457)
('\nE_new: ', -136.44106535619457, '\tE_old: ', -136.27747155920372, '\tDE_elst: ', -58.713942627239845)
################################
Число итераций 4; Энергия 0.267931478289; Время (мин) 0
################################



...

('atom: ', 19.954250348929499, ' ind: ', 1, ' sum: ', 20.594250348929499)
('Minimum of parabola: ', 19.620045725140898)
('Candidates: ', 179.55044646519829, '\n\t', 176.78980444312788, '\n\t', 49.646361409828529)
('E_new: ', 193.74861142427108, ' E_old: ', 52.406506329339464)
('Supposed minimum: ', 193.74861142427108)
Parabola is not at minimum.
('Actual minimum: ', 49.646361409828529)
('\nE_new: ', 49.646361409828529, '\tE_old: ', 52.406506329339464, '\tDE_elst: ', -12.250565853940564)
################################
Число итераций 1197; Энергия 27133.7082068; Время (мин) 4
################################

('atom: ', 20.56, ' ind: ', -1, ' sum: ', 19.919999999999998)
('atom: ', 20.56, ' ind: ', 0, ' sum: ', 20.559999999999999)
('atom: ', 20.56, ' ind: ', 1, ' sum: ', 21.199999999999999)
('Minimum of parabola: ', 20.239536346528467)
('Candidates: ', 1984.1772526798379, '\n\t', 1983.6188193298369, '\n\t', 1212.2318121519852)
('E_new: ', 2080.2518100063899, ' E_old: ', 1212.75666061558

('atom: ', 16.953878154640964, ' ind: ', -1, ' sum: ', 16.313878154640964)
('atom: ', 16.953878154640964, ' ind: ', 0, ' sum: ', 16.953878154640964)
('atom: ', 16.953878154640964, ' ind: ', 1, ' sum: ', 17.593878154640965)
('Minimum of parabola: ', 16.633423436521319)
('Candidates: ', 274.21633003858824, '\n\t', 274.11405644731065, '\n\t', 130.06524628203971)
('E_new: ', 292.15854664686049, ' E_old: ', 130.10739154851106)
('Supposed minimum: ', 292.15854664686049)
Parabola is not at minimum.
('Actual minimum: ', 130.06524628203971)
('\nE_new: ', 130.06524628203971, '\tE_old: ', 130.10739154851106, '\tDE_elst: ', 1.6283526583413033)
################################
Число итераций 1199; Энергия 27134.2752006; Время (мин) 4
################################



('atom: ', 21.328, ' ind: ', -1, ' sum: ', 20.687999999999999)
('atom: ', 21.328, ' ind: ', 0, ' sum: ', 21.327999999999999)
('atom: ', 21.328, ' ind: ', 1, ' sum: ', 21.968)
('Minimum of parabola: ', 21.007937687626242)
('Candidates: ', 2456.4291886987512, '\n\t', 2456.3206414762394, '\n\t', 1341.3416747190422)
('E_new: ', 2595.7337228142424, ' E_old: ', 1341.2676876855467)
('Supposed minimum: ', 2595.7337228142424)
Parabola is not at minimum.
('Actual minimum: ', 1341.3416747190422)
('\nE_new: ', 1341.3416747190422, '\tE_old: ', 1341.2676876855467, '\tDE_elst: ', 23.609409619016446)
################################
Число итераций 1200; Энергия 27134.2752006; Время (мин) 4
################################



('atom: ', 28.127378820100049, ' ind: ', -1, ' sum: ', 27.487378820100048)
('atom: ', 28.127378820100049, ' ind: ', 0, ' sum: ', 28.127378820100049)
('atom: ', 28.127378820100049, ' ind: ', 1, ' sum: ', 28.767378820100049)
('Minimum of parabola: ', 27.807180719250066)
('Candidates: ', 3613.4412319084718, '\n\t', 3615.9064672552022, '\n\t', 11582.752570664936)
('E_new: ', 2619.1258595250547, ' E_old: ', 11580.093075057663)
('Supposed minimum: ', 2619.1258595250547)
('\nE_new: ', 2619.1258595250547, '\tE_old: ', 11580.093075057663, '\tDE_elst: ', 70.698687833880484)
################################
Число итераций 1201; Энергия 36095.2424161; Время (мин) 4
################################

('atom: ', 9.653, ' ind: ', -1, ' sum: ', 9.0129999999999999)
('atom: ', 9.653, ' ind: ', 0, ' sum: ', 9.6530000000000005)


('atom: ', 9.653, ' ind: ', 1, ' sum: ', 10.293000000000001)
('Minimum of parabola: ', 9.3348795071483419)
('Candidates: ', 207.02967998890546, '\n\t', 207.4536571233956, '\n\t', 63.50714526028348)
('E_new: ', 225.28860223411357, ' E_old: ', 63.032331079783319)
('Supposed minimum: ', 225.28860223411357)
Parabola is not at minimum.
('Actual minimum: ', 63.50714526028348)
('\nE_new: ', 63.50714526028348, '\tE_old: ', 63.032331079783319, '\tDE_elst: ', 7.0740856127879281)
################################
Число итераций 1202; Энергия 36095.2424161; Время (мин) 4
################################

('atom: ', 18.295, ' ind: ', -1, ' sum: ', 17.655000000000001)
('atom: ', 18.295, ' ind: ', 0, ' sum: ', 18.295000000000002)
('atom: ', 18.295, ' ind: ', 1, ' sum: ', 18.935000000000002)


In [1]:
import matplotlib.pyplot as plt

mmas = - np.array(mas)

plt.plot(tt, mmas)
plt.xlabel(r"Время работы программы в минутах")
plt.ylabel(r"$\Delta E$")
plt.grid(True)
# plt.interactive(False)
plt.show()

ImportError: No module named matplotlib.pyplot

In [40]:
num = 280
coor = 2
h = 0.005
M = 100
mmm = []
ccc = []

print("Energy: ", E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, num, nghb_d, nghb_a, nghb_b, nghb_nb, full=True))

# print(E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh,nm))
for k in range(0, M):
    atoms[nm][coor] = atoms[nm][coor] + h
    EE = E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, 
           num, nghb_d, nghb_a, nghb_b, nghb_nb, full=True) 
    # E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh,nm)
    mmm.append(EE)
    ccc.append(k*h)  

import matplotlib.pyplot as plt
plt.plot(ccc, mmm)
plt.ylabel(r"Изменение энергии")
plt.xlabel(r"Изменение положения атома")
plt.grid(True)
plt.show()
    
print(E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, num, nghb_d, nghb_a, nghb_b, nghb_nb, full=True))

# print(E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh,nm))

0.5463493965623099


NameError: name 'nm' is not defined

In [14]:
print(E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, 2))

6.90602313923


In [47]:
print(E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, 140))
atoms[140][0] = atoms[140][0] + 0.5
print(E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, 140))

4438.13048459
4438.13048459


In [59]:
a = np.array([1, 2, 3])
b = np.array([3, 2, 1])
print(a*b)

[3 4 3]


In [74]:
c = 10*[1, 1, 1]
#c[2][3]=[4,5]

In [77]:
print(c[5][2])

TypeError: 'int' object is not subscriptable

In [101]:
def ff(f=True):
    if f:
        print("True")
    else:
        print("False")
      
        
ff(f=True)

True


In [97]:
c = [1, 2, 3]
[c1, c2, c3] = c

print(c2)

2


In [102]:
for k in range(-2, 2):
    print(k)

-2
-1
0
1


In [107]:
print(L)

5


In [238]:
time_start = millis()
# число атомов в изучаемом белке
l = len(atoms)
# размеры шагов по ортам в ангстремах
h = [0.02, 0.02, 0.02]
# сколько шагов делаем вдоль каждого орта с учетом знака
L = 5
S = 0
# размер окрестности атома, по которой честно суммируе
r = 10
atoms_old = atoms
record = [0, 0, 0]
num = 434  # pyrandom.randrange(0, l)
nghb_d = nghb_dihedrals(dihedrals, num)
nghb_a = nghb_angles(angles, num)
nghb_b = nghb_bonds(bonds, num)
nghb_nb = nghbnb(atoms, num, r)[0]
nghbc = nghbnb(atoms, num, r)[1]
[Cx, Cy, Cz] = coef_nb(atoms, num, nghbc)

DE_elst = E_elst(atoms, neighbours, two_bonds_neigh, three_bonds_neigh, num) - \
          E_elst_m(atoms, neighbours, two_bonds_neigh, three_bonds_neigh, num, nghb_nb)

E_old = E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, 
          num, nghb_d, nghb_a, nghb_b, nghb_nb, full=True)

E_record = E_old
for px in range(-L, L + 1):
    for py in range(-L, L + 1):
        for pz in range(-L, L + 1):
            atoms[num][1] = atoms_old[num][1] + px * h[0]
            atoms[num][2] = atoms_old[num][2] + py * h[1]
            atoms[num][3] = atoms_old[num][3] + pz * h[2]
            E_new = DE_elst + Cx * px * h[0] + Cy * py * h[1] + Cz * pz * h[2] + E(atoms, dihedrals, angles, bonds,
                                                                                   neighbours, two_bonds_neigh,
                                                                                   three_bonds_neigh, num, nghb_d,
                                                                                   nghb_a, nghb_b, nghb_nb, full=False)
            #  "-0.05" чтобы не учитвать погрешности в аппроксимации
            if E_new < E_record - 0.05:  
                record = [px, py, pz]
                E_record = E_new
atoms = atoms_old
atoms[num][1] = atoms_old[num][1] + record[0] * h[0]
atoms[num][2] = atoms_old[num][2] + record[1] * h[1]
atoms[num][3] = atoms_old[num][3] + record[2] * h[2]

time_now = millis()
tm = (time_now - time_start) / 60000

ETRUE = E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, three_bonds_neigh, num, nghb_d, nghb_a, nghb_b,
          nghb_nb, full=True)
DE = ETRUE - E_old

print("Энергия {}; Время (мин) {}".format(DE, tm))
k = 0
prp_atm(atoms, k)
os.system('./to-pdb atoms_out' + str(k) + '.csv output' + str(k) + '.pdb')

Энергия -169.7640675101076; Время (мин) 0.04665


0

In [229]:
print(nghb_d)
print(nghb_a)
print(nghb_b)
print(nghb_nb)
# print(nghbc)
print([Cx, Cy, Cz])

[998, 999, 1000, 1076, 1077, 1078, 1086, 1087, 1088, 1274, 1275, 1276, 1297, 1298, 1299, 1329, 1330, 1331, 1347, 1348, 1349]
[692, 738, 739, 746, 747, 757, 876, 877, 878, 896, 916, 931]
[420, 421, 422, 497]
[98, 112, 114, 125, 132, 162, 163, 167, 168, 172, 174, 176, 177, 188, 194, 198, 202, 203, 204, 206, 207, 208, 209, 210, 211, 215, 217, 218, 223, 225, 226, 230, 233, 236, 239, 243, 255, 257, 258, 259, 261, 263, 267, 268, 270, 271, 272, 273, 276, 278, 280, 282, 283, 286, 288, 291, 294, 295, 298, 300, 301, 303, 304, 305, 309, 310, 311, 312, 313, 317, 319, 320, 321, 324, 325, 327, 328, 331, 332, 333, 339, 340, 341, 343, 344, 346, 348, 349, 351, 354, 355, 360, 362, 365, 368, 371, 373, 376, 379, 382, 384, 386, 387, 391, 392, 393, 396, 399, 400, 402, 404, 405, 406, 407, 408, 410, 412, 415, 416, 422, 424, 425, 426, 428, 431, 432, 434, 436, 437, 438, 439, 441, 444, 449, 453, 454, 471, 472, 474, 475, 478, 482, 487, 490, 492, 496, 498, 503, 504, 509, 511, 512, 513, 514, 515, 516, 517, 520, 521

In [164]:
def dgdfg(nghbc):
    E = np.array([0, 0, 0])
    for j in nghbc:
        r = np.array([1, 1, 1])
        E[0] = E[0] + r[0]
        E[1] = E[1] + r[1]
        E[2] = E[2] + r[2]
    return E


nghbc = [1, 2, 3, 4, 76]
print(dgdfg(nghbc))


[5 5 5]


In [212]:
E = np.array([0.0, 0.0, 0.0])
print(E[0])

i = 5
j = 8
r = np.array(atoms[j][1:4]) - np.array(atoms[i][1:4])
ccc = atoms[i][6] * atoms[j][6] * (atoms[j][1] - atoms[i][1]) / pow(np.linalg.norm(r), 3)
print(ccc)

E[0] = E[0] + ccc 
print(E[0])
print(E[0] * 1389.38757)
print(ccc * 1389.38757)

0.0
1.63332281568e-05
1.63332281568e-05
0.022693184179
0.022693184179


In [175]:
print(
    (E[0] + atoms[i][6] * atoms[j][6] * (atoms[j][1] - atoms[i][1]) / pow(np.linalg.norm(r), 3)) 
    * 
    1389.38757
)

0.022693184179


In [210]:
y = np.array([0.0, 0.0, 0.0])
y[1] = y[1] + ccc
print(y)


[  0.00000000e+00   1.63332282e-05   0.00000000e+00]


In [239]:
num = 434
Delta = E_vdw(atoms, neighbours, two_bonds_neigh, three_bonds_neigh, num) - \
        E_vdw_m(atoms, neighbours, two_bonds_neigh, three_bonds_neigh, num, nghb_nb)

#print(nghb_nb)
print (Delta)

-0.0945943788217


In [249]:
aa = DE_elst + \
     Cx*px*h[0] + Cy*py*h[1] + Cz*pz*h[2] + \
     E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, 
       three_bonds_neigh, num, nghb_d, nghb_a, nghb_b, nghb_nb, full=False)

In [250]:
bb = E(atoms, dihedrals, angles, bonds, neighbours, two_bonds_neigh, 
       three_bonds_neigh, num, nghb_d, nghb_a, nghb_b, nghb_nb, full=True)

In [254]:
bb - aa

1.0280167218129463

In [252]:
DE_elst

3.2317934304534361